In [1]:
import networkx as nx
import scipy as sp
import numpy as np
import random
import copy
from scipy import sparse
from numpy import linalg
from networkx.algorithms.community.label_propagation import label_propagation_communities
import collections

%matplotlib inline

In [2]:
political=nx.read_gml("polblogs.gml",label='id')

#make the graph directed with no parallels
political = nx.DiGraph(political)

#returns a set of nodes of the biggest connected component
largest = max(nx.connected_components(political.to_undirected()), key=len)

# creates the graph of the biggest connected component including direction
CC_max = political.subgraph(largest)


In [25]:
a = CC_max.copy()

convergence_factor = 0.01

#get the b% of values
percentage=0.1

#Returns a list with the value of the nodes
original_values = dict(CC_max.nodes(data='value'))

numbers_of_items_kept_from_percentage = round(len(a)*percentage)

# shuffle the dictionary of values to get the random nodes

a1 = list(original_values.items())
np.random.shuffle(a1)

final_dictionary = dict(a1)

#keep the first percentage b% of the nodes

nodes_kept = list(final_dictionary.items())[:numbers_of_items_kept_from_percentage]

list_to_be_deleted = list(dict(nodes_kept).keys())

nodes_to_discover = list(final_dictionary.items())[numbers_of_items_kept_from_percentage:]

#initialize not known node values
        
for key in dict(nodes_kept).keys():
    final_dictionary[key] = 0.5


od = collections.OrderedDict(sorted(final_dictionary.items()))


previous_values = np.matrix(list(od.values()))
previous_values = np.reshape(previous_values,(np.size(previous_values,1), 1))

keys_order = list(od.keys())

new_values = np.matrix(list(od.values()))
new_values = np.reshape(new_values,(np.size(new_values,1), 1))

# adjacency matrix ordered according to the nodes inside keys_order

A = nx.adjacency_matrix(CC_max ,nodelist = keys_order)

sum_of_neighbors = np.sum(A,axis=1)

size_of_vector = np.size(sum_of_neighbors,0)

sum_of_neighbors = np.reshape(sum_of_neighbors,(size_of_vector, 1))

indexes_to_be_deleted = np.where(np.isin(A,list_to_be_deleted))

np.delete(A,indexes_to_be_deleted, axis=0)


#the order holds
#print(sum_of_neighbors)
#print(CC_max[keys_order[0]])

while True:
    new_values = A * new_values
    new_values = new_values/sum_of_neighbors
    new_values[np.isnan(new_values)] = 0

    norm_of_previous_values = linalg.norm(np.array(np.reshape(previous_values,(1, size_of_vector))),1)
    
    norm_new = linalg.norm(np.array(np.reshape(new_values,(1, size_of_vector))),1)
    
    #print(norm_of_previous_values)
    #print(norm_new)
    
    norm_difference = abs(norm_new - norm_of_previous_values)

    print(norm_difference)
    break
    if norm_difference < convergence_factor:
        break

      
            
            


1.0
1.0
0.0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: in the future the special handling of scalars will be removed from delete and raise an error
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide
